# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다.
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다.

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ?] <br>
모델이 예측한 가장 적합한 단어 : 잠입 <br>
당신의 답변과 모델 예측의 유사도 : 0.34 <br>
아쉽네요. 더 생각해보세요.

# 파일 한 개

In [177]:
# 텍스트 전처리

from lxml import etree      # xml, html 파일을 처리하기 위한 경량화된 라이브러리, etree : xml파일을 파싱하고 조작할 수 있는 함수 
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import pandas as pd
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec, FastText

In [178]:
# BOBR210001636718.json 파일 경로
file_path = 'data/BOBR210001636718.json'

# JSON 파일 불러오기
with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# 현재 json_data의 최상위 키들을 출력해봅니다.
print("JSON 데이터의 최상위 키:", json_data.keys())

# 만약 'text' 키가 없다면, 오류가 발생합니다.
# ['header', 'text', 'SJML'] 와 같이 'text'가 포함되어 있는지 확인하세요.
# 만약 있다면, 오류는 다른 곳에서 발생했을 수 있습니다.

JSON 데이터의 최상위 키: dict_keys(['SJML'])


In [179]:
file_path = './data/BOBR210001636718.json'

with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

sentences = [item['content'] for item in json_data['SJML']['text']]

okt = Okt()
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords
ko_stopwords = load_stopwords('ko_stopwords.txt')

preprocessed_data = []

for sentence in sentences:
    #nouns = okt.morphs(sentence)
    tokens = okt.morphs(sentence, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    preprocessed_data.append(tokens)

preprocessed_data[:5]

[['이수근', '은', '너무', '화', '내다', '너무', '시끄럽다', ','],
 ['백',
  '번',
  '만',
  '번',
  '돈까스',
  '가격',
  '책정',
  '은',
  '이수근',
  '맞다',
  '봄',
  '.',
  '진짜',
  '분명',
  '서다',
  '400',
  '2만',
  '인데',
  '는',
  '(',
  '반',
  '사회',
  '적용',
  ')',
  '이네',
  '이르다',
  '욕',
  '하다',
  '사람',
  '분명하다',
  '생기다',
  ','],
 ['왤케', '싸우다', '임'],
 ['그냥',
  '소스',
  '조그마하다',
  '그릇',
  '담기다',
  '부어',
  '먹다',
  '찍다',
  '먹다',
  '알다',
  '하다',
  '않다',
  ','],
 ['44만원', '식', '재료', '판매', '수익', '22만원', ',']]

In [180]:
model = FastText(
    sentences=preprocessed_data,
    vector_size=50,
    window=5,
    min_count=1, 
    sg=1
)

model.wv.vectors.shape

(2711, 50)

In [181]:
model.wv.most_similar('짬뽕')

[('래', 0.16803424060344696),
 ('세넼', 0.1608595997095108),
 ('햇반', 0.14180752635002136),
 ('슴', 0.13784340023994446),
 ('맞네욭', 0.13614748418331146),
 ('파네', 0.13567087054252625),
 ('차차', 0.13547828793525696),
 ('천재', 0.12999187409877777),
 ('맨', 0.12988676130771637),
 ('랰', 0.12741051614284515)]

In [182]:
import random

all_words = list(model.wv.index_to_key)
    
while True:
    a_word, b_word = random.sample(all_words, 2)
    similarity = model.wv.similarity(a_word, b_word)

    if similarity >= 0.8:
        print(f'A: {a_word}, B: {b_word}')
        break

A: 안전, B: 직전


In [183]:
candidate_c_words = [word for word in all_words if word not in [a_word, b_word]]

c_word = random.choice(candidate_c_words)
analogy_result = model.wv.most_similar(positive=[c_word, a_word], negative=[b_word], topn=1)

predicted_d_word = analogy_result[0][0]

print(f'관계 [{a_word} : {b_word} = {c_word} : ?]')

user_input = input('모델이 예측한 가장 적합한 단어를 입력하세요: ').strip()

print(f'\n모델이 예측한 가장 적합한 단어: {predicted_d_word}')

similarity = model.wv.similarity(user_input, predicted_d_word)
print(f'당신의 답 {user_input} 과 모델의 예측 유사도 : {similarity:.4f}')
if similarity < 0.5:
    print('아쉽네요. 더 생각해보세요.')
else:
    print('거의 정답이예요!')


관계 [안전 : 직전 = 배댕 : ?]

모델이 예측한 가장 적합한 단어: 골
당신의 답 후진 과 모델의 예측 유사도 : 0.0802
아쉽네요. 더 생각해보세요.


---

# 더 많은 파일로 생성

In [184]:
import json
import glob
from gensim.models import FastText
from konlpy.tag import Okt

# data/media 폴더 안에 있는 json 파일 목록 가져오기
files = glob.glob("data/media/*.json")

# 앞에서 100개만 사용
sample_files = files[:100]

sentences = []

#for fname in sample_files:
#    with open(fname, "r", encoding="utf-8") as f:
#        data = json.load(f)
#    sentence.extend([item['content'] for item in data]['SJML']['text']) -> 오류 발생, 구조가 이렇게 안 생긴 파일도 있는듯

for file in sample_files:
    try:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

        text_list = data['SJML']['text']

        # 리스트의 각 아이템을 확인
        for item in text_list:
            # item이 딕셔너리(dictionary) 타입이고, 'content' 키가 있는지 확인
            if isinstance(item, dict) and 'content' in item:
                sentences.append(item['content'])
            else:
                pass

    except Exception as e:
        continue

okt = Okt()

# 불용어 처리
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords
ko_stopwords = load_stopwords('ko_stopwords.txt')

preprocessed_data = []

korean_stopword = re.compile('[^가-힣ㄱ-ㅎㅏ-ㅣ.,?!:;]')

for sentence in sentences:
    filtered_sentence = korean_stopword.sub('', sentence)
    tokens = okt.morphs(filtered_sentence, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    preprocessed_data.append(tokens)



In [ ]:
#nouns_per_sentence = [
#    [w for w,t in okt.pos(" ".join(s), norm=True, stem=True) if t=='Noun']
#    for s in preprocessed_data
#]
#nouns_per_sentence = [ns for ns in nouns_per_sentence if ns] 

In [185]:
model = FastText(
    sentences=preprocessed_data,
    vector_size=50,
    window=5,
    min_count=1, 
    sg=1
)

model.wv.vectors.shape

(14474, 50)

In [167]:
model.wv.most_similar('짬뽕')

[('고다이로', 0.9978534579277039),
 ('류현상', 0.9978373050689697),
 ('오존층', 0.9977177977561951),
 ('근무시간', 0.9976955652236938),
 ('타임머신', 0.9973708987236023),
 ('무료음원', 0.9972454309463501),
 ('돼지고기', 0.9972369074821472),
 ('위아래', 0.9972060918807983),
 ('믹스테잎', 0.9971486330032349),
 ('파이터', 0.9970571994781494)]

In [186]:
import random

all_words = list(model.wv.index_to_key)
    
while True:
    a_word, b_word = random.sample(all_words, 2)
    similarity = model.wv.similarity(a_word, b_word)

    if similarity >= 0.8:
        print(f'A: {a_word}, B: {b_word}')
        break

A: 뮤직, B: 보안


In [187]:
candidate_c_words = [word for word in all_words if word not in [a_word, b_word]]

c_word = random.choice(candidate_c_words)
analogy_result = model.wv.most_similar(positive=[c_word, a_word], negative=[b_word], topn=1)

predicted_d_word = analogy_result[0][0]

print(f'관계 [{a_word} : {b_word} = {c_word} : ?]')

user_input = input('모델이 예측한 가장 적합한 단어를 입력하세요: ').strip()

print(f'\n모델이 예측한 가장 적합한 단어: {predicted_d_word}')

similarity = model.wv.similarity(user_input, predicted_d_word)
print(f'당신의 답 {user_input} 과 모델의 예측 유사도 : {similarity:.4f}')
if similarity < 0.5:
    print('아쉽네요. 더 생각해보세요.')
else:
    print('거의 정답이예요!')


관계 [뮤직 : 보안 = 반성 : ?]

모델이 예측한 가장 적합한 단어: 고명
당신의 답 안녕 과 모델의 예측 유사도 : 0.9818
거의 정답이예요!
